In [84]:
import requests  ## used for requesting to access the HTML webpage
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import bs4 ## used for web scrapping
from bs4 import BeautifulSoup 
import datetime ## used for working with datetime formats
import re
from seaborn import set_style
set_style('whitegrid')
from time import sleep
import warnings
warnings.filterwarnings("ignore")

# Get recent data from playbill.com

Get weekly data from **08-05-2022 to 2023-05-14.**

An example url for playbill is https://www.playbill.com/grosses?week=2023-02-19.
We can see that at each url we can find data in a table and the link is formatted as `https://www.playbill.com/grosses?week="`+ `date` where `date` represents the last day of the week (Sunday) and it is formatted as year-month-day.

In [85]:
start = datetime.datetime.strptime("2022-05-08", "%Y-%m-%d")
end = datetime.datetime.strptime("2023-05-14", "%Y-%m-%d")
dates_to_scrape = list(pd.date_range(start, end, freq='W'))

In [86]:
dates_to_scrape[0]

Timestamp('2022-05-08 00:00:00', freq='W-SUN')

In [87]:
dates_to_scrape  = [(str(date_timestamp)).split()[0] for date_timestamp in dates_to_scrape]

In [88]:
len(dates_to_scrape)

54

In [89]:
dates_to_scrape[0]

'2022-05-08'

# Scraping begins..

In [90]:
show_names=[]
potential_gross=[]
seats_sold=[]
previews=[]
cap=[]
diff=[]
diff_cap=[]
top_ticket=[]
avg_ticket=[]
seats_in_theatre=[]
theatre_name=[]
dates = []

In [91]:
base_url = "https://www.playbill.com/grosses?week="
for date in dates_to_scrape: ## Iteration for every week in dates_to_scrape (08-05-2022 to 2023-05-14)
    
    url = base_url + str(date)
    print(str(date) + ":" + url)
    
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser') ##Using Beautiful Soup from bs4 package to read the webpage
    table = soup.find('tbody')  ## soup.find searches for the table from webpage
    rows = table.find_all('tr') ## searches rows (tr is the HTML code for rows)
    for row in rows:
        
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]    ## Scrapping values
        
        dates.append(date)
        show_names.append(cols[0].split('\n')[0])
        potential_gross.append(cols[1])
        diff.append(cols[2])
        avg_ticket.append(cols[3].split('\n')[0])
        seats_sold.append(cols[4].split('\n')[0])
        seats_in_theatre.append(cols[4].split('\n')[1])
        previews.append(cols[5].split('\n')[0])
        cap.append(cols[6])
        diff_cap.append(cols[7])

2022-05-08:https://www.playbill.com/grosses?week=2022-05-08
2022-05-15:https://www.playbill.com/grosses?week=2022-05-15
2022-05-22:https://www.playbill.com/grosses?week=2022-05-22
2022-05-29:https://www.playbill.com/grosses?week=2022-05-29
2022-06-05:https://www.playbill.com/grosses?week=2022-06-05
2022-06-12:https://www.playbill.com/grosses?week=2022-06-12
2022-06-19:https://www.playbill.com/grosses?week=2022-06-19
2022-06-26:https://www.playbill.com/grosses?week=2022-06-26
2022-07-03:https://www.playbill.com/grosses?week=2022-07-03
2022-07-10:https://www.playbill.com/grosses?week=2022-07-10
2022-07-17:https://www.playbill.com/grosses?week=2022-07-17
2022-07-24:https://www.playbill.com/grosses?week=2022-07-24
2022-07-31:https://www.playbill.com/grosses?week=2022-07-31
2022-08-07:https://www.playbill.com/grosses?week=2022-08-07
2022-08-14:https://www.playbill.com/grosses?week=2022-08-14
2022-08-21:https://www.playbill.com/grosses?week=2022-08-21
2022-08-28:https://www.playbill.com/gros

In [128]:
## Having scrapped all the relevant data, converting the data into a dataframe
df = pd.DataFrame({'Date': dates,
                   'Show_name':show_names, 
                   'Potential_Gross':potential_gross,
                   'Difference':diff,
                   'Average_ticket':avg_ticket,
                   'Seats_Sold':seats_sold,
                   'Seats_in_theater':seats_in_theatre,
                   'Previews':previews,
                   '%cap':cap,
                   'diff_cap':diff_cap})

In [129]:
df.head()

,Date,Show_name,Potential_Gross,Difference,Average_ticket,Seats_Sold,Seats_in_theater,Previews,%cap,diff_cap
0,2022-05-08,Aladdin,"$1,091,385.55","$125,858.37",$83.17,"13,123","1,727",8,94.98%,4.39%
1,2022-05-08,American Buffalo,"$514,501.63","-$14,344.67",$109.80,"4,686",751,8,78.00%,0.94%
2,2022-05-08,Beetlejuice,"$900,622.50","$24,883.90",$111.99,"8,042","1,602",8,62.75%,5.51%
3,2022-05-08,Birthday Candles,"$264,076.00","-$22,311.00",$72.87,"3,624",727,8,62.31%,-5.16%
4,2022-05-08,The Book of Mormon,"$905,752.71","$48,571.74",$115.02,"7,875","1,066",8,92.34%,6.61%


# Data Cleaning

In [130]:
type(df['Average_ticket'][0])

str

In [131]:
df.replace(',','', regex=True, inplace=True)  ## Removing commas into the strings
df.replace('%','', regex=True, inplace=True)  ## Removing the % sign into the strings

df['Average_ticket']=df['Average_ticket'].str.replace('$', '')
df["Seats_Sold"] = pd.to_numeric(df["Seats_Sold"], downcast="float")
df["Seats_in_theater"] = pd.to_numeric(df["Seats_in_theater"], downcast="float")
df["Previews"] = pd.to_numeric(df["Previews"], downcast="float")
df["%cap"] = pd.to_numeric(df["%cap"], downcast="float")
df["diff_cap"] = pd.to_numeric(df["diff_cap"], downcast="float")

df["Potential_Gross"]= df["Potential_Gross"].str.replace('$', '')
df["Potential_Gross"] = pd.to_numeric(df["Potential_Gross"], downcast="float")

df["Difference"]= df["Difference"].str.replace('$', '')

In [132]:
df.head(2)

,Date,Show_name,Potential_Gross,Difference,Average_ticket,Seats_Sold,Seats_in_theater,Previews,%cap,diff_cap
0,2022-05-08,Aladdin,1091385.55,125858.37,83.17,13123.0,1727.0,8.0,94.980003,4.39
1,2022-05-08,American Buffalo,514501.63,-14344.67,109.80,4686.0,751.0,8.0,78.000000,0.94


# Save data in a csv file

In [133]:
df.to_csv('../recent_54_weeks_data.csv', index=False)